In [1]:
import pandas as pd
import numpy as np

In [2]:
X_train = pd.read_csv("D:/Hackathons/GST_GOV/Train_60/X_Train_Data_Input_Imputed.csv")
y_train = pd.read_csv('D:/Hackathons/GST_GOV/Train_60/Y_Train_Data_Target.csv')
X_test = pd.read_csv('D:/Hackathons/GST_GOV/Test_20/X_Test_Data_Input_Imputed.csv')
y_test = pd.read_csv('D:/Hackathons/GST_GOV/Test_20/Y_Test_Data_Target.csv')

In [3]:
y_train = y_train.drop('ID', axis=1)
y_test = y_test.drop('ID', axis=1)

In [4]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(785133, 21)
(785133, 1)
(261712, 21)
(261712, 1)


In [5]:
import pandas as pd
import pickle
import json
import os
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             roc_auc_score, roc_curve, confusion_matrix, log_loss, balanced_accuracy_score)
from sklearn.ensemble import (RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, StackingClassifier, VotingClassifier)
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import HistGradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.neural_network import MLPClassifier
from imblearn.ensemble import (BalancedRandomForestClassifier, EasyEnsembleClassifier, RUSBoostClassifier, BalancedBaggingClassifier)
from sklearn.ensemble import VotingClassifier
import gc

# Create a directory to store model and plots
os.makedirs('model_plots', exist_ok=True)
os.makedirs('models', exist_ok=True)

# List of classifiers including imblearn models
classifiers = {
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Naive Bayes': GaussianNB(),
    'XGBoost': XGBClassifier(),
    'LightGBM': LGBMClassifier(),
    'CatBoost': CatBoostClassifier(verbose=0),  # Silent output
    'HistGradientBoosting': HistGradientBoostingClassifier(),
    'Extra Trees': ExtraTreesClassifier(),
    # Voting
    'Voting Classifier': VotingClassifier(estimators=[
        ('lr', LogisticRegression()),
        ('rf', RandomForestClassifier()),
        ('xgb', XGBClassifier())
    ], voting='soft')
}

# Define dictionary to store model performance metrics and plots
metrics_dict = {}

# Helper function to plot AUC-ROC curve with metrics
def plot_roc_curve(fpr, tpr, auc_roc, name, accuracy, precision, recall, f1):
    plt.figure()
    plt.plot(fpr, tpr, label=f'AUC-ROC (AUC = {auc_roc:.4f})')
    plt.plot([0, 1], [0, 1], 'k--', label='Random Guessing')
    plt.xlabel('False Positive Rate (FPR)')
    plt.ylabel('True Positive Rate (TPR)')
    plt.title(f'AUC-ROC Curve - {name}\n'
              f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}')
    plt.legend(loc='lower right')
    plt.grid(True)
    plt.savefig(f'model_plots/{name}_roc_curve.png')
    plt.close()

# Helper function to plot confusion matrix with metrics
def plot_confusion_matrix(cm, name, accuracy, precision, recall, f1):
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.title(f'Confusion Matrix - {name}\n'
              f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.savefig(f'model_plots/{name}_confusion_matrix.png')
    plt.close()

# Train, predict, and store metrics for each model
for name, clf in classifiers.items():
    print(f"Training {name}...")
    
    start_time = time.time()  # Start time tracking
    
    # Fit the model
    clf.fit(X_train, y_train)
    
    # Predict probabilities for ROC and log loss
    y_prob = clf.predict_proba(X_test)[:, 1]
    
    # Predictions
    y_pred = clf.predict(X_test)
    
    # Calculate all metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc_roc = roc_auc_score(y_test, y_prob)
    cm = confusion_matrix(y_test, y_pred)
    log_loss_value = log_loss(y_test, y_prob)
    balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
    
    # ROC curve metrics
    fpr, tpr, thresholds = roc_curve(y_test, y_prob)
    
    # Plot AUC-ROC and Confusion Matrix with metrics, save the plots
    plot_roc_curve(fpr, tpr, auc_roc, name, accuracy, precision, recall, f1)
    plot_confusion_matrix(cm, name, accuracy, precision, recall, f1)
    
    # Save the model using pickle
    with open(f'models/{name}.pkl', 'wb') as model_file:
        pickle.dump(clf, model_file)
    
    # Store the results in a dictionary
    metrics_dict[name] = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc_roc': auc_roc,
        'confusion_matrix': cm.tolist(),  # Convert to list for JSON
        'log_loss': log_loss_value,
        'balanced_accuracy': balanced_accuracy,
        'fpr': fpr.tolist(),
        'tpr': tpr.tolist(),
        'thresholds': thresholds.tolist(),
        'hyperparameters': clf.get_params(),
        'roc_curve_plot': f'model_plots/{name}_roc_curve.png',
        'confusion_matrix_plot': f'model_plots/{name}_confusion_matrix.png',
        'training_time': time.time() - start_time
    }

    # Print metrics and training time
    print(f"{name} training complete.")
    print(f"Metrics for {name}:\n"
          f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}, AUC: {auc_roc:.4f}")
    print(f"Confusion Matrix:\n{cm}")
    print(f"Training time: {metrics_dict[name]['training_time']:.2f} seconds\n")

    # Clear memory for large objects and empty cache
    del clf, y_prob, y_pred
    gc.collect()

# Save the metrics to a JSON file
with open('classification_metrics.json', 'w') as f:
    json.dump(metrics_dict, f)

print("Model training, evaluation, and saving complete.")


Training Logistic Regression...


d:\Python\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\Python\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression training complete.
Metrics for Logistic Regression:
Accuracy: 0.9675, Precision: 0.8104, Recall: 0.8559, F1: 0.8325, AUC: 0.9832
Confusion Matrix:
[[232092   4942]
 [  3556  21122]]
Training time: 8.41 seconds

Training Decision Tree...
Decision Tree training complete.
Metrics for Decision Tree:
Accuracy: 0.9677, Precision: 0.8297, Recall: 0.8268, F1: 0.8283, AUC: 0.9090
Confusion Matrix:
[[232847   4187]
 [  4274  20404]]
Training time: 2.56 seconds

Training Random Forest...


d:\Python\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Random Forest training complete.
Metrics for Random Forest:
Accuracy: 0.9770, Precision: 0.8449, Recall: 0.9265, F1: 0.8838, AUC: 0.9939
Confusion Matrix:
[[232838   4196]
 [  1815  22863]]
Training time: 124.60 seconds

Training Gradient Boosting...


d:\Python\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Gradient Boosting training complete.
Metrics for Gradient Boosting:
Accuracy: 0.9759, Precision: 0.8210, Recall: 0.9517, F1: 0.8815, AUC: 0.9936
Confusion Matrix:
[[231913   5121]
 [  1192  23486]]
Training time: 221.34 seconds

Training AdaBoost...


d:\Python\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\Python\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoost training complete.
Metrics for AdaBoost:
Accuracy: 0.9748, Precision: 0.8223, Recall: 0.9348, F1: 0.8749, AUC: 0.9930
Confusion Matrix:
[[232049   4985]
 [  1610  23068]]
Training time: 73.93 seconds

Training K-Nearest Neighbors...


d:\Python\Lib\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
